In [23]:
from zope.interface import Interface, implementer
from zope.interface.verify import verifyClass
import attr
import sys

```{=latex}
\begin{frame}
```

In [42]:
class IRectangle(Interface):
    def get_height() -> float:
        """Return height"""
    def get_width() -> float:
        """Return width"""
    def set_height(height: float):
        """Set height"""
    def set_width(width: float):
        """Set width"""

```{=latex}
\end{frame}
```

In [45]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    side: float

    def get_height(self) -> float:
        return side
    
    def get_width(self) -> float:
        return side
try:
    verifyClass(IRectangle, Square)
except Exception as exc:
    print(str(exc).split(":")[1], file=sys.stderr)


    The __main__.IRectangle.set_height(height) attribute was not provided
    The __main__.IRectangle.set_width(width) attribute was not provided


In [56]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float

    def get_height(self) -> float:
        return self._side
    
    def get_width(self) -> float:
        return self._side

    def set_height(self, height: float):
        self._side = height # ???
        
    def set_width(self, width: float):
        self._side = width # ???

In [57]:
def area(rectangle: Rectangle) -> float:
    return rectangle.get_height() * rectangle.get_width()

def double_height(rectangle: Rectangle) -> float:
    rectangle.set_height(2 * rectangle.get_height())

x = Square(side=5)
print(area(x))
double_height(x)
print(area(x))

25
100


In [54]:
class IRectangle(Interface):
    def get_height() -> float:
        """Return height"""
    def get_width() -> float:
        """Return width"""
    def with_height(height: float) -> IRectangle:
        """Rectangle with same width, new height"""
    def with_width(width: float) -> IRectangle:
        """Rectangle with same height, new width"""

In [60]:
@implementer(IRectangle)
@attr.s(auto_attribs=True, frozen=True)
class Rectangle:
    _height: float
    _width: float
    
    def get_height(self) -> float:
        return self._height
    
    def get_width(self) -> float:
        return self._width

    def with_height(self, height) -> float:
        return attr.evolve(self, height=height)
    
    def with_width(self, width) -> float:
        return attr.evolve(self, width=width)

In [61]:
@implementer(IRectangle)
@attr.s(auto_attribs=True)
class Square:
    _side: float

    def get_height(self) -> float:
        return self._side
    
    def get_width(self) -> float:
        return self._side

    def with_height(self, height: float) -> IRectangle:
        return Rectangle(width=self._side, height=height)
    
    def with_width(self, width: float) -> IRectangle:
        return Rectangle(height=self._side, width=width)

try:
    verifyClass(IRectangle, Square)
except Exception as exc:
    print(str(exc).split(":")[1], file=sys.stderr)

In [62]:
def area(rectangle):
    return rectangle.get_height() * rectangle.get_width()

def double_height(rectangle):
    return rectangle.with_height(2 * rectangle.get_height())

x = Square(side=5)
print(area(x))
print(area(double_height(x)))

25
50


In [1]:
def sum_with_extra(e1, e2, things=[]):
    things.append(e1)
    things.append(e2)
    return sum(things)

In [2]:
sum_with_extra(1, 2, [3, 4])

10

In [3]:
sum_with_extra(1, 2)

3

In [4]:
# Whoops!
sum_with_extra(1, 2)

6

In [7]:
def sum_with_extra_v2(e1, e2, things=None):
    if things is None:
        things = []
    things.append(e1)
    things.append(e2)
    return sum(things)

In [8]:
sum_with_extra_v2(1, 2, [3, 4])

10

In [9]:
sum_with_extra_v2(1, 2)

3

In [10]:
sum_with_extra_v2(1, 2)

3

In [12]:
things = [1, 2]
sum_with_extra_v2(1, 2, things)

6

In [13]:
sum_with_extra_v2(1, 2, things)

9

In [14]:
def sum_with_extra_v3(e1, e2, things=None):
    if things is None:
        things = []
    things = things.copy()
    things.append(e1)
    things.append(e2)
    return sum(things)

In [15]:
from pyrsistent import v

In [16]:
def sum_with_extra_p_v1(e1, e2, things=v()):
    things = things.append(e1)
    things = things.append(e2)
    return sum(things)

In [17]:
sum_with_extra_p_v1(1, 2)

3

In [19]:
sum_with_extra_p_v1(1, 2)

3

In [21]:
things = v(1, 2)
sum_with_extra_p_v1(1, 2, things)

6

In [22]:
sum_with_extra_p_v1(1, 2, things)

6